In [132]:
#For web scraping
from bs4 import BeautifulSoup
import requests

#For data
import pandas as pd
from pandas import Series,DataFrame

In [140]:
#url to read the html file
url = 'http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/'

# Request content from web page
result = requests.get(url)
c = result.content

#We should split the content result into a list of two string elements 
#because tags <pre> were previously outside of html tags
c2 = c.split("</html>\n")

# Set as Beautiful Soup Object
soup = BeautifulSoup(c2[1])

#url to download (example)
#http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/1028-20100710-hne.tgz

**As it's displayed below, beautiful soup doesn't get "pre" tags without a split**

In [66]:
BeautifulSoup(c)

<html><head><title>VoxForge Repository</title>\n<style type="text/css">\n\t.siteFunctions {\n\t\ttext-align: right;\n\t}\n\t.copyright {\n\t\ttest-align: left;\n\t\tcolor: #2E3436;\n\t\tfont-family: sans-serif;\n                font-size: small;\n\t}\n\n\tbody {\n\t\tfont-family: "DejaVu Sans", "Lucida Sans Unicode", sans-serif;\n\t\tfont-weight:\tnormal;\n\t\tword-spacing:\tnormal;\n\t\tletter-spacing:\tnormal; \n\t\ttext-transform:\tnone;\n\t\tfont-size: medium;\n                text-align: justify;\n\t}\n\th2 {\n\t\tfont-size:\t1.5em;\n\t\tfont-weight:\t700;\n\t\tmargin-top:1em;\n\t\tmargin-bottom:0.8em;\n\t}\n\th3 {\n\t\tfont-size:\t1.1em;\n\t\tfont-weight:\t600;\n\t\tmargin-top:1em;\n\t\tmargin-bottom:0.4em;\n\t}\n\tp, ol, ul {\n\t\tfont-size:\t1em;\n\t\tmargin-top:0.4em;\n\t\tmargin-bottom:0.4em;\n\t}\t\n\t.heading {\n\t\tbackground-color: #555753;\n                color: #D3D7CF;\n\t\tfont-size: 40px;\n\t\tvertical-align: bottom;\n\t}\n\t.logo {\n\t\twidth: 100px; \n\t\tfloat: l

In [112]:
#Check types to see if we are still aligned
print (type(c))
print type(c2[1])

<type 'str'>
<type 'str'>


In [141]:
#Extract both download links and informations about (date and size)

sub_link = []
information = []

for link in soup.find_all('a'):
    if '.tgz' in link.get('href'):
        sub_link.append(link.get('href'))
        information.append(link.next_sibling)   

#Check if we have same amount of elements in both lists 
print(len(information))
print len(sub_link)

6244
6244


In [142]:
#Cleaning content of information list

size = []

for i in range (len(information)):
    information[i] = " ".join(information[i].split())
    information[i] = str(information[i])
    size.append(information[i][-4:])
    information[i] = information[i][:-4]
    
print (information[5000])
print (size[5000])

2016-05-01 13:05 
1.7M


In [143]:
#Create a dataframe to analyze
d = {'sub_link':sub_link,'date':information, 'size':size}

scraping_df = pd.DataFrame(d)
scraping_df['date'] = pd.to_datetime(scraping_df['date'])
scraping_df.head()

,date,size,sub_link
0,2010-07-20 09:47:00,1.3M,1028-20100710-hne.tgz
1,2017-03-24 04:09:00,1.1M,1337ad-20170321-ajg.tgz
2,2017-03-24 04:09:00,1.1M,1337ad-20170321-tkg.tgz
3,2017-03-24 04:09:00,1.0M,1337ad-20170321-ynk.tgz
4,2012-05-11 04:52:00,1.5M,1snoke-20120412-hge.tgz


In [144]:
scraping_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6244 entries, 0 to 6243
Data columns (total 3 columns):
date        6244 non-null datetime64[ns]
size        6244 non-null object
sub_link    6244 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 146.4+ KB


In [146]:
scraping_df.to_csv('/Users/kevenlemoing/Sites/sandvik_code_assignement/data/scraping_result.csv')

In [ ]:

for i in range (len(scraping_df['size'])):
    scraping_df['size'][i] = scraping_df['size'][i][:-1]
scraping_df['size'][1]

/Users/kevenlemoing/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [148]:
scraping_df['size'] = scraping_df['size'][:-1]

scraping_df['size'] = scraping_df['size'].astype(int)

ValueError: invalid literal for long() with base 10: '1.3M'